In [17]:
import tkinter as tk
# TODO: Combine all three GUI into one GUI (File import, County BUS, Checkbox)
# Create the main window
root = tk.Tk()
root.title("Checkbox Example")

# Variable to store the checkbox state
checkbox_AUX = tk.BooleanVar()
checkbox_N_2 = tk.BooleanVar()
checkbox_N_1_1 = tk.BooleanVar()
checkbox_PO = tk.BooleanVar()

title = f"Check the files you want to save during the process. \n If none of the options are checked, \n only maximum violation csv file will be saved"

label = tk.Label(root, text=title)
label.pack(anchor=tk.W)
# Create a checkbox

checkbox1 = tk.Checkbutton(root, text="Save AUX File", variable=checkbox_AUX)
checkbox1.pack(anchor="w")

checkbox2 = tk.Checkbutton(root, text="Save N-2 case and result File", variable=checkbox_N_2)
checkbox2.pack(anchor="w")

checkbox3 = tk.Checkbutton(root, text="Save N-1-1 case and result File", variable=checkbox_N_1_1)
checkbox3.pack(anchor="w")

checkbox4 = tk.Checkbutton(root, text="Save PO cases", variable=checkbox_PO)
checkbox4.pack(anchor="w")

# Function to display the checkbox state
def show_selection():
    selected = []

    selected.append(f"Option AUX = {checkbox_AUX.get()}")

    selected.append(f"Option N_2 = {checkbox_N_2.get()}")

    selected.append(f"Option N_1_1 = {checkbox_N_1_1.get()}")

    selected.append(f"Option PO = {checkbox_PO.get()}")
    print("Selected:", ", ".join(selected))
        
# Function to close the window
def close_window():
    root.destroy()


# Button to check the state
submit_button = tk.Button(root, text="Submit", command=close_window)
submit_button.pack()


# Run the application
root.mainloop()

checkbox_AUX.get(), checkbox_N_2.get(), checkbox_N_1_1.get(), checkbox_PO.get()

(False, False, False, False)

In [18]:
import tkinter as tk
from tkinter import ttk

def county_bus_selector(county_list, bus_number_list):
    selected = {'county': '', 'bus': ''}

    def update_suggestions(entry_var, listbox, data_list):
        typed = entry_var.get()
        listbox.delete(0, tk.END)
        if typed:
            suggestions = [item for item in data_list if typed.lower() in item.lower()]
            for item in suggestions:
                listbox.insert(tk.END, item)

    def on_select(event, entry_var, listbox):
        selection = listbox.get(listbox.curselection())
        entry_var.set(selection)
        listbox.delete(0, tk.END)

    def confirm_selection():
        selected['county'] = county_var.get()
        selected['bus'] = bus_var.get()
        root.destroy()

    root = tk.Tk()
    root.title("County and Bus Selector")
    root.geometry("450x350")

    # County input
    ttk.Label(root, text="Enter County Name:").pack(pady=(10, 2))
    county_var = tk.StringVar()
    county_entry = ttk.Entry(root, textvariable=county_var, width=40)
    county_entry.pack()

    county_listbox = tk.Listbox(root, height=5)
    county_listbox.pack(pady=(0, 10))
    county_listbox.bind("<<ListboxSelect>>", lambda e: on_select(e, county_var, county_listbox))

    county_var.trace_add("write", lambda *args: update_suggestions(county_var, county_listbox, county_list))

    # Bus number input
    ttk.Label(root, text="Enter Bus Number:").pack(pady=(5, 2))
    bus_var = tk.StringVar()
    bus_entry = ttk.Entry(root, textvariable=bus_var, width=40)
    bus_entry.pack()

    bus_listbox = tk.Listbox(root, height=5)
    bus_listbox.pack(pady=(0, 10))
    bus_listbox.bind("<<ListboxSelect>>", lambda e: on_select(e, bus_var, bus_listbox))

    bus_var.trace_add("write", lambda *args: update_suggestions(bus_var, bus_listbox, bus_number_list))

    # Confirm button
    confirm_btn = ttk.Button(root, text="Confirm", command=confirm_selection)
    confirm_btn.pack(pady=15)

    root.mainloop()

    return selected['county'], selected['bus']

# Example usage
if __name__ == "__main__":
    counties = ["Travis", "Harris", "Dallas", "Bexar", "Montgomery", "Tarrant", "El Paso"]
    buses = ["10123", "10058", "20012", "30077", "40022", "50033"]
    selected_county, selected_bus = county_bus_selector(counties, buses)
    print("User selected county:", selected_county)
    print("User selected bus number:", selected_bus)


User selected county: 
User selected bus number: 


In [23]:
import tkinter as tk
from tkinter import ttk
from tkinterdnd2 import DND_FILES, TkinterDnD
import os
import string

# $ RTP Team's request:
# $ 1. Add another GUI for County and additional Bus number input.
# $ 2. Instead of display Option A B C, have a string box for naming. Default = Option A B C
# $ 3. Check box GUI user interface for output save from each function. Default = everything unchecked. (Only save the result of the M.O)
class FileDropApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Drag and Drop File Uploader")

        self.frame = ttk.Frame(root, padding=10)
        self.frame.pack(fill=tk.BOTH, expand=True)

        self.option_counter = 0
        self.listboxes = []
        self.result = None  # Store result to return later

        self.add_option_box()

        # Add buttons
        btn_frame = ttk.Frame(self.frame)
        btn_frame.pack(pady=10)

        add_btn = ttk.Button(btn_frame, text="Add Option Box", command=self.add_option_box)
        add_btn.pack(side=tk.LEFT, padx=5)

        finish_btn = ttk.Button(btn_frame, text="Finish Upload", command=self.handle_finish_upload)
        finish_btn.pack(side=tk.LEFT, padx=5)

    def add_option_box(self):
        # Generate default title like "Option A", "Option B", etc.
        default_title = f"Option {string.ascii_uppercase[self.option_counter]}"
        self.option_counter += 1

        name_frame = ttk.Frame(self.frame)
        name_frame.pack(anchor=tk.W, pady=(10, 0))

        name_label = ttk.Label(name_frame, text="Option Name:")
        name_label.pack(side=tk.LEFT)

        name_entry = ttk.Entry(name_frame, width=30)
        name_entry.insert(0, default_title)  # Pre-fill with default name
        name_entry.pack(side=tk.LEFT, padx=5)

        def confirm_name():
            title = name_entry.get().strip()
            if not title:
                messagebox.showerror("Input Error", "Please enter a name for the option.")
                return
            if any(existing_title == title for existing_title, _ in self.listboxes):
                messagebox.showerror("Duplicate Name", f"The option name '{title}' is already used.")
                return

            # Display as section label
            label = ttk.Label(self.frame, text=title)
            label.pack(anchor=tk.W)

            listbox = tk.Listbox(self.frame, selectmode=tk.MULTIPLE, width=80, height=5)
            listbox.pack(pady=5, fill=tk.X)

            listbox.drop_target_register(DND_FILES)
            listbox.dnd_bind('<<Drop>>', lambda e, lb=listbox: self.on_drop(e, lb))

            self.listboxes.append((title, listbox))

            # Lock the name entry after confirmation
            name_entry.config(state='disabled')
            confirm_btn.config(state='disabled')

        confirm_btn = ttk.Button(name_frame, text="Confirm", command=confirm_name)
        confirm_btn.pack(side=tk.LEFT, padx=5)



    def on_drop(self, event, listbox):
        files = self.root.tk.splitlist(event.data)
        for file in files:
            if os.path.isfile(file) and file not in listbox.get(0, tk.END):
                listbox.insert(tk.END, file)

    def handle_finish_upload(self):
        result = []
        for title, lb in self.listboxes:
            file_list = [lb.get(i) for i in range(lb.size())]
            result.append((title, file_list))

        self.result = result
        self.root.quit()  # Close the Tkinter loop but keep the app object

def run_app():
    root = TkinterDnD.Tk()
    app = FileDropApp(root)
    root.mainloop()
    root.destroy()  # Fully close window
    return app.result  # Return the result

# Example of calling the app and using the result
# ! Check the content inside the box, if user upload other files than PWB or AUX file or upload more than one PWB or AUX file. Raise an error and
# ! reset the box
if __name__ == "__main__":
    uploaded_files = run_app()
    print("\nReturned Data Structure:")
    for option, files in uploaded_files:
        print(f"{option}:")
        for f in files:
            print(f"  {f}")
        if not files:
            print("  [No files uploaded]")
    
    print(uploaded_files[0][0],uploaded_files[0][1])


TclError: can't invoke "destroy" command: application has been destroyed

In [23]:
import os

file = r"O:\Project\Project vault\Baytown\3wl4_Opt2A_Maint.pwb"
direcotry = os.path.dirname(file) + "\\"
direcotry

'O:\\Project\\Project vault\\Baytown\\'

In [32]:
import tkinter as tk
from tkinter import ttk, messagebox
from tkinterdnd2 import TkinterDnD, DND_FILES
import os
import string

class MergedApp:
    def __init__(self, root):
        self.root = root
        self.root.title("RTP Tool")

        self.notebook = ttk.Notebook(root)
        self.notebook.pack(fill=tk.BOTH, expand=True)

        # Shared data
        self.uploaded_files = []
        self.selected_county = ''
        self.selected_bus = ''
        self.checkbox_states = {
            'AUX': tk.BooleanVar(),
            'N_2': tk.BooleanVar(),
            'N_1_1': tk.BooleanVar(),
            'PO': tk.BooleanVar()
        }

        # Tabs
        self.file_tab = ttk.Frame(self.notebook)
        self.county_tab = ttk.Frame(self.notebook)
        self.checkbox_tab = ttk.Frame(self.notebook)

        self.notebook.add(self.file_tab, text='File Upload')
        self.notebook.add(self.county_tab, text='County & Bus')
        self.notebook.add(self.checkbox_tab, text='Save Options')

        self.init_file_tab()
        self.init_county_tab()
        self.init_checkbox_tab()

    def init_file_tab(self):
        self.option_counter = 0
        self.listboxes = []
        self.file_frame = ttk.Frame(self.file_tab, padding=10)
        self.file_frame.pack(fill=tk.BOTH, expand=True)

        self.add_option_box()

        btn_frame = ttk.Frame(self.file_frame)
        btn_frame.pack(pady=10)

        ttk.Button(btn_frame, text="Add Option Box", command=self.add_option_box).pack(side=tk.LEFT, padx=5)
        ttk.Button(btn_frame, text="Finish Upload", command=self.handle_finish_upload).pack(side=tk.LEFT, padx=5)

    def add_option_box(self):
        default_title = f"Option {string.ascii_uppercase[self.option_counter]}"
        self.option_counter += 1

        name_frame = ttk.Frame(self.file_frame)
        name_frame.pack(anchor=tk.W, pady=(10, 0))

        name_label = ttk.Label(name_frame, text="Option Name:")
        name_label.pack(side=tk.LEFT)

        name_entry = ttk.Entry(name_frame, width=30)
        name_entry.insert(0, default_title)
        name_entry.pack(side=tk.LEFT, padx=5)

        def confirm_name():
            title = name_entry.get().strip()
            if not title:
                messagebox.showerror("Input Error", "Please enter a name for the option.")
                return
            if any(existing_title == title for existing_title, _ in self.listboxes):
                messagebox.showerror("Duplicate Name", f"The option name '{title}' is already used.")
                return

            label = ttk.Label(self.file_frame, text=title)
            label.pack(anchor=tk.W)

            listbox = tk.Listbox(self.file_frame, selectmode=tk.MULTIPLE, width=80, height=5)
            listbox.pack(pady=5, fill=tk.X)

            listbox.drop_target_register(DND_FILES)
            listbox.dnd_bind('<<Drop>>', lambda e, lb=listbox: self.on_drop(e, lb))

            self.listboxes.append((title, listbox))

            name_entry.config(state='disabled')
            confirm_btn.config(state='disabled')

        confirm_btn = ttk.Button(name_frame, text="Confirm", command=confirm_name)
        confirm_btn.pack(side=tk.LEFT, padx=5)

    def on_drop(self, event, listbox):
        files = self.root.tk.splitlist(event.data)

        # Get current files already in the listbox
        current_files = listbox.get(0, tk.END)
        all_files = list(current_files)

        for file in files:
            if os.path.isfile(file) and file not in all_files:
                all_files.append(file)

        # Count file types
        ext_counts = {}
        for file in all_files:
            _, ext = os.path.splitext(file)
            ext = ext.lower()
            ext_counts[ext] = ext_counts.get(ext, 0) + 1

        # Check for duplicates
        duplicate_exts = [ext for ext, count in ext_counts.items() if count > 1 and ext in [".pwb", ".aux"]]

        if duplicate_exts:
            msg = f"You uploaded more than one file with the following extension(s): {', '.join(duplicate_exts)}.\n" \
                "Only one .pwb and one .aux file are allowed per option.\n\n" \
                "Would you like to clear the files from this box?"

            # Show error window with Yes/No options
            if messagebox.askyesno("Duplicate File Types", msg):
                listbox.delete(0, tk.END)  # Clear files
            return

        # If all good, insert new files
        for file in files:
            if file not in current_files:
                listbox.insert(tk.END, file)


    def handle_finish_upload(self):
        self.uploaded_files = []
        for title, lb in self.listboxes:
            file_list = [lb.get(i) for i in range(lb.size())]
            self.uploaded_files.append((title, file_list))
        messagebox.showinfo("Upload Complete", "File selections recorded.")

    def init_county_tab(self):
        counties = ["Travis", "Harris", "Dallas", "Bexar", "Montgomery", "Tarrant", "El Paso"]
        buses = ["10123", "10058", "20012", "30077", "40022", "50033"]

        def update_suggestions(entry_var, listbox, data_list):
            typed = entry_var.get().split(",")[-1].strip()
            listbox.delete(0, tk.END)
            if typed:
                suggestions = [item for item in data_list if typed.lower() in item.lower()]
                for item in suggestions:
                    listbox.insert(tk.END, item)

        def on_select(event, entry_var, listbox):
            selection_indices = listbox.curselection()
            if not selection_indices:
                return
            selection = listbox.get(selection_indices[0])

            # Get the current input
            current = entry_var.get()
            parts = current.split(',')

            # Replace the last part (in-progress typed input)
            parts[-1] = selection

            # Rebuild the string and set it
            new_text = ', '.join(part.strip() for part in parts if part.strip())
            entry_var.set(new_text)

            listbox.delete(0, tk.END)

        def confirm_selection():
            self.selected_county = [c.strip() for c in county_var.get().split(",") if c.strip()]
            self.selected_bus = [b.strip() for b in bus_var.get().split(",") if b.strip()]

            # Validate input
            invalid_counties = [c for c in self.selected_county if c not in counties]
            invalid_buses = [b for b in self.selected_bus if b not in buses]

            if invalid_counties or invalid_buses:
                msg = ""
                if invalid_counties:
                    msg += f"Invalid counties: {', '.join(invalid_counties)}\n"
                if invalid_buses:
                    msg += f"Invalid buses: {', '.join(invalid_buses)}"
                messagebox.showerror("Invalid Entry", msg)
                return

            messagebox.showinfo(
                "Selection Confirmed",
                f"Counties: {', '.join(self.selected_county)}\nBuses: {', '.join(self.selected_bus)}"
            )

        county_var = tk.StringVar()
        bus_var = tk.StringVar()

        ttk.Label(self.county_tab, text="Enter County Name(s) (comma separated):").pack(pady=(10, 2))
        county_entry = ttk.Entry(self.county_tab, textvariable=county_var, width=40)
        county_entry.pack()

        county_listbox = tk.Listbox(self.county_tab, height=5)
        county_listbox.pack()
        county_listbox.bind("<<ListboxSelect>>", lambda e: on_select(e, county_var, county_listbox))
        county_var.trace_add("write", lambda *args: update_suggestions(county_var, county_listbox, counties))

        ttk.Label(self.county_tab, text="Enter Bus Number(s) (comma separated):").pack(pady=(10, 2))
        bus_entry = ttk.Entry(self.county_tab, textvariable=bus_var, width=40)
        bus_entry.pack()

        bus_listbox = tk.Listbox(self.county_tab, height=5)
        bus_listbox.pack()
        bus_listbox.bind("<<ListboxSelect>>", lambda e: on_select(e, bus_var, bus_listbox))
        bus_var.trace_add("write", lambda *args: update_suggestions(bus_var, bus_listbox, buses))

        ttk.Button(self.county_tab, text="Confirm", command=confirm_selection).pack(pady=15)


    def init_checkbox_tab(self):
        tk.Label(
            self.checkbox_tab,
            text="Check the files you want to save during the process.\n"
                 "If none are checked, only the maximum violation CSV file will be saved."
        ).pack(anchor=tk.W, pady=10)

        for key in self.checkbox_states:
            tk.Checkbutton(
                self.checkbox_tab,
                text=f"Save {key.replace('_', '-') if key != 'PO' else 'PO cases'}",
                variable=self.checkbox_states[key]
            ).pack(anchor=tk.W)

        ttk.Button(self.checkbox_tab, text="Submit", command=self.close_app).pack(pady=10)

    def close_app(self):
        self.root.quit()

def run_merged_gui():
    root = TkinterDnD.Tk()
    app = MergedApp(root)
    root.mainloop()
    root.destroy()
    return app.uploaded_files, app.selected_county, app.selected_bus, {k: v.get() for k, v in app.checkbox_states.items()}


if __name__ == "__main__":
    files, county, bus, options = run_merged_gui()
    print("Uploaded Files:", files)
    print("Selected County:", county)
    print("Selected Bus:", bus)
    print("Checkbox Options:", options)


Uploaded Files: []
Selected County: ['Travis', 'Harris']
Selected Bus: ['10058', '20012']
Checkbox Options: {'AUX': True, 'N_2': True, 'N_1_1': True, 'PO': False}
